In [38]:
import pandas as pd 
import numpy as np 
import os 
import re
import sys 
import ast 
import json

In [39]:
results_dir = "../../../european-city-data/rag-sustainability/results/results-combined_prompts/"
folders = os.listdir(results_dir)
results_df = pd.read_csv(f"{results_dir}/recommended_cities.csv")
sim_df = pd.read_csv(f"{results_dir}/context_response_similarity_scores.csv")

cities_df = pd.read_csv("../../../european-city-data/cities/worldcities.csv")
cities = list(cities_df['city'])

eucities_df = pd.read_csv("../../../european-city-data/archive/city_abstracts_embeddings.csv")
eu_cities = list(cities_df['city'])

sar_results_dir = "../../../european-city-data/rag-sustainability/results/results-combined_prompts_SAR/"
sar_results = pd.read_csv(f"{sar_results_dir}/recommended_cities_sar.csv")

sim_sar = pd.read_csv(f"{sar_results_dir}/context_response_similarity_scores.csv")

In [40]:
combined_df = pd.merge(
    left=results_df, 
    right=sar_results,
    how='left',
    left_on=['model', 'prompt_id'],
    right_on=['model', 'prompt_id'],
)

In [41]:
llama = results_df[results_df['model'] == 'llama3point1-instruct']
mistral = results_df[results_df['model'] == 'mistral-instruct']

In [42]:
llama

,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable
0,llama3point1-instruct,prompt_17_gemini-ui,['Kaunas'],I recommend Kaunas in Lithuania for your expe...,['Vitoria-Gasteiz'],I recommend Vitoria-Gasteiz because it is a c...
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"I recommend Copenhagen, Denmark because it is...",['Copenhagen'],I recommend Copenhagen and why I recommended ...
2,llama3point1-instruct,prompt_42_gpt-4o-mini,['Amsterdam'],"I recommend Amsterdam, Netherlands. Amsterdam...","['Nalchik', 'Varna', 'Kahramanmaras']","I recommend Nalchik, Russia because of its be..."
3,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Kaunas', 'Thessaloniki', 'Strasbourg']","I recommend Kaunas, Lithuania. I recommend Ka...","['Kaunas', 'Oradea', 'Strasbourg', 'Thessaloni...","I recommend Kaunas, Lithuania because it is a..."
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,['Kaunas'],I recommend Kaunas for your medieval history ...,['Kaunas'],"I recommend Kaunas, Lithuania because it has ..."
...,...,...,...,...,...,...
195,llama3point1-instruct,prompt_29_gpt-4o-mini,['Innsbruck'],"I recommend Innsbruck, Austria. Innsbruck is ...",['Innsbruck'],"I recommend Innsbruck, Austria, for the best ..."
196,llama3point1-instruct,prompt_44_gemini-ui,['Stavanger'],"I recommend Stavanger, Norway, because it off...",['Stavanger'],"I recommend Stavanger, Norway because of its ..."
197,llama3point1-instruct,prompt_59_gpt-4o-mini,"['Munich', 'Thessaloniki', 'Arkhangelsk', 'Gaz...","I recommend Munich, Germany for its vibrant a...","['Munich', 'Thessaloniki', 'Gaziantep', 'Malat...",I recommend Munich and why I recommended it: ...
198,llama3point1-instruct,prompt_50_gemini-ui,"['Varna', 'Tallinn', 'London', 'Cork', 'Innsbr...","I recommend Varna, Bulgaria for your weekend ...",['Varna'],I recommend Varna because it has a great bala...


### What are the sustainability ranks of the recommended cities?

In [43]:
def get_ranks(df):
    city_s_ranks = []
    for i, row in df.iterrows():
        context_path = os.path.join(results_dir, row['model'], row['prompt_id'], 'cities_sustainable.json')
        with open(context_path) as f:
            context = json.load(f)

        city_rank = []
        for city in ast.literal_eval(row['rec_cities_sustainable']):
            for j, rec_city in enumerate(context): 
                if city == rec_city['city']:
                    city_rank.append({
                        'city': city,
                        's-fairness-rank': j+1
                    })
        city_s_ranks.append(city_rank)

    df['sustainability_ranks'] = city_s_ranks
    return df


In [44]:
llama_ranks = get_ranks(llama)
llama_ranks

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_38824/742151116.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sustainability_ranks'] = city_s_ranks


,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,sustainability_ranks
0,llama3point1-instruct,prompt_17_gemini-ui,['Kaunas'],I recommend Kaunas in Lithuania for your expe...,['Vitoria-Gasteiz'],I recommend Vitoria-Gasteiz because it is a c...,"[{'city': 'Vitoria-Gasteiz', 's-fairness-rank'..."
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"I recommend Copenhagen, Denmark because it is...",['Copenhagen'],I recommend Copenhagen and why I recommended ...,"[{'city': 'Copenhagen', 's-fairness-rank': 9}]"
2,llama3point1-instruct,prompt_42_gpt-4o-mini,['Amsterdam'],"I recommend Amsterdam, Netherlands. Amsterdam...","['Nalchik', 'Varna', 'Kahramanmaras']","I recommend Nalchik, Russia because of its be...","[{'city': 'Nalchik', 's-fairness-rank': 10}, {..."
3,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Kaunas', 'Thessaloniki', 'Strasbourg']","I recommend Kaunas, Lithuania. I recommend Ka...","['Kaunas', 'Oradea', 'Strasbourg', 'Thessaloni...","I recommend Kaunas, Lithuania because it is a...","[{'city': 'Kaunas', 's-fairness-rank': 2}, {'c..."
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,['Kaunas'],I recommend Kaunas for your medieval history ...,['Kaunas'],"I recommend Kaunas, Lithuania because it has ...","[{'city': 'Kaunas', 's-fairness-rank': 6}]"
...,...,...,...,...,...,...,...
195,llama3point1-instruct,prompt_29_gpt-4o-mini,['Innsbruck'],"I recommend Innsbruck, Austria. Innsbruck is ...",['Innsbruck'],"I recommend Innsbruck, Austria, for the best ...","[{'city': 'Innsbruck', 's-fairness-rank': 6}]"
196,llama3point1-instruct,prompt_44_gemini-ui,['Stavanger'],"I recommend Stavanger, Norway, because it off...",['Stavanger'],"I recommend Stavanger, Norway because of its ...","[{'city': 'Stavanger', 's-fairness-rank': 4}]"
197,llama3point1-instruct,prompt_59_gpt-4o-mini,"['Munich', 'Thessaloniki', 'Arkhangelsk', 'Gaz...","I recommend Munich, Germany for its vibrant a...","['Munich', 'Thessaloniki', 'Gaziantep', 'Malat...",I recommend Munich and why I recommended it: ...,"[{'city': 'Munich', 's-fairness-rank': 5}, {'c..."
198,llama3point1-instruct,prompt_50_gemini-ui,"['Varna', 'Tallinn', 'London', 'Cork', 'Innsbr...","I recommend Varna, Bulgaria for your weekend ...",['Varna'],I recommend Varna because it has a great bala...,"[{'city': 'Varna', 's-fairness-rank': 4}]"


In [45]:
mistral_ranks = get_ranks(mistral)
mistral_ranks

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_38824/742151116.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sustainability_ranks'] = city_s_ranks


,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,sustainability_ranks
200,mistral-instruct,prompt_17_gemini-ui,['Budapest'],1. Paris is a city in France. Here is some inf...,"['Budapest', 'Copenhagen']","1. Budapest, Hungary\n2. Copenhagen, Denmark\n...","[{'city': 'Budapest', 's-fairness-rank': 8}, {..."
201,mistral-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"1. I recommend Copenhagen, Denmark. Copenhagen...","['Copenhagen', 'Coimbra']","1. Copenhagen, Denmark - Copenhagen is a city ...","[{'city': 'Copenhagen', 's-fairness-rank': 9},..."
202,mistral-instruct,prompt_42_gpt-4o-mini,"['Amsterdam', 'Mykolaiv', 'Craiova']","1. Amsterdam, Netherlands\n2. Istanbul, Turkey...",['Amsterdam'],"1. ""What are the best things to do in Amsterda...","[{'city': 'Amsterdam', 's-fairness-rank': 2}]"
203,mistral-instruct,prompt_0_gpt-4o-mini,['Sibiu'],1. I recommend Sibiu in Romania.\n\nSibiu is a...,['Sibiu'],1. I recommend Sibiu in Romania.\n\nSibiu is a...,"[{'city': 'Sibiu', 's-fairness-rank': 9}]"
204,mistral-instruct,prompt_44_gemini-1.5-pro-001,[],1. {{see\n| name | alt | url | email\n| addres...,['Prague'],"1. Option 1: Prague, Czech Republic\n* {{marke...","[{'city': 'Prague', 's-fairness-rank': 9}]"
...,...,...,...,...,...,...,...
395,mistral-instruct,prompt_29_gpt-4o-mini,"['Innsbruck', 'Varna', 'Kaunas', 'Kayseri', 'I...","1. I recommend recommending Innsbruck, Austria...","['Innsbruck', 'Kaunas', 'Varna', 'Kayseri', 'I...","1. Innsbruck, Austria\n2. Kaunas, Lithuania\n3...","[{'city': 'Innsbruck', 's-fairness-rank': 6}, ..."
396,mistral-instruct,prompt_44_gemini-ui,[],1. The capital of the Czech Republic is Prague...,['Arkhangelsk'],1. What is the sustainability score for Arkhan...,"[{'city': 'Arkhangelsk', 's-fairness-rank': 1}]"
397,mistral-instruct,prompt_59_gpt-4o-mini,"['Malatya', 'Varna', 'Mykolaiv', 'Munich', 'Ar...","1. I recommend visiting Malatya, Turkey. The c...","['Mykolaiv', 'Munich', 'Varna', 'Craiova', 'Ar...",1. I recommend visiting Mykolaiv in Ukraine. M...,"[{'city': 'Mykolaiv', 's-fairness-rank': 7}, {..."
398,mistral-instruct,prompt_50_gemini-ui,[],1. Option 1:\n\n* {{eat\n| name\n| url\n| emai...,"['Innsbruck', 'Antalya', 'Cork', 'London']",1. The sustainability score for the city of In...,"[{'city': 'Innsbruck', 's-fairness-rank': 6}, ..."


Number of prompts where the first city has lowest s-fairness: 

In [46]:
def is_lowest_rank(ranked_list):
    if len(ranked_list):
        min_rank = min(ranked_list, key=lambda x: x['s-fairness-rank'])['s-fairness-rank']

        if ranked_list[0]['s-fairness-rank'] == min_rank:
            return 1
    return 0

In [47]:
llama_min_ranks = sum(llama_ranks['sustainability_ranks'].apply(is_lowest_rank))
llama_min_ranks

141

In [48]:
mistral_min_ranks = sum(mistral_ranks['sustainability_ranks'].apply(is_lowest_rank))
mistral_min_ranks

87

Llama outperforms Mistral (2x better) here as well: in 72.5% of the prompts, the top choice has the lowest s-fairness compared to the rest of the recommended cities, NOT retrieved cities (as opposed to 43.5% for Mistral)

In [49]:
def is_top_choice_most_sustainable(ranked_list):
    if len(ranked_list):
        if ranked_list[0]['s-fairness-rank'] == 1:
            return 1
    return 0

In [50]:
l_top_choice = sum(llama_ranks['sustainability_ranks'].apply(is_top_choice_most_sustainable))
m_top_choice = sum(mistral_ranks['sustainability_ranks'].apply(is_top_choice_most_sustainable))

In [51]:
print(l_top_choice, m_top_choice)

8 20


In [52]:
def most_sustainable(ranked_list):
    if len(ranked_list):
        if ranked_list[0]['s-fairness-rank'] == 1:
            return True
    return False

In [53]:
l_sar = llama_ranks[(llama_ranks['sustainability_ranks'].apply(most_sustainable))]
l_sar

,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,sustainability_ranks
39,llama3point1-instruct,prompt_47_gemini-ui,"['Thessaloniki', 'Arkhangelsk']","I recommend Thessaloniki, Greece, because it ...","['Thessaloniki', 'Krasnodar', 'Kars']",I recommend Thessaloniki because it offers a ...,"[{'city': 'Thessaloniki', 's-fairness-rank': 1..."
116,llama3point1-instruct,prompt_23_gpt-4o-mini,['Kazan'],"I recommend Kazan, Russia because it offers a...",['Munich'],"I recommend Munich, Germany. I recommend Muni...","[{'city': 'Munich', 's-fairness-rank': 1}]"
123,llama3point1-instruct,prompt_26_gpt-4o-mini,"['Strasbourg', 'Plovdiv', 'Maastricht']","I recommend Strasbourg, France, because it is...",['Strasbourg'],I recommend Strasbourg because it's a popular...,"[{'city': 'Strasbourg', 's-fairness-rank': 1}]"
137,llama3point1-instruct,prompt_48_gemini-ui,['Milan'],"I recommend Milan, Italy. I recommend Milan b...",['Milan'],I recommend Milan because it has good transpo...,"[{'city': 'Milan', 's-fairness-rank': 1}]"
158,llama3point1-instruct,prompt_38_gemini-ui,"['Milan', 'Hamburg']",I recommend Milan and Hamburg because they ha...,['Milan'],I recommend Milan as the best city for your b...,"[{'city': 'Milan', 's-fairness-rank': 1}]"
162,llama3point1-instruct,prompt_67_gpt-4o-mini,['Bologna'],"I recommend Bologna, Italy, for its rich culi...",['Bologna'],"I recommend Bologna, Italy for a culinary exp...","[{'city': 'Bologna', 's-fairness-rank': 1}]"
168,llama3point1-instruct,prompt_35_gemini-1.5-pro-001,['Istanbul'],I recommend Istanbul as the city for stargazi...,"['Mykolaiv', 'Arkhangelsk', 'Istanbul']",I recommend Mykolaiv and why you recommended ...,"[{'city': 'Mykolaiv', 's-fairness-rank': 1}, {..."
173,llama3point1-instruct,prompt_63_gemini-ui,['Nicosia'],"I recommend Nicosia, Cyprus in August. Nicosi...",['Nicosia'],I recommend Nicosia in Cyprus because it offe...,"[{'city': 'Nicosia', 's-fairness-rank': 1}]"


In [54]:
m_sar = mistral_ranks[(mistral_ranks['sustainability_ranks'].apply(most_sustainable))]
m_sar

,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,sustainability_ranks
229,mistral-instruct,prompt_10_gpt-4o-mini,['Arkhangelsk'],"1. I recommend Arkhangelsk, Russia. Arkhangels...",['Arkhangelsk'],"1. I recommend Arkhangelsk, Russia. Arkhangels...","[{'city': 'Arkhangelsk', 's-fairness-rank': 1}]"
239,mistral-instruct,prompt_47_gemini-ui,[],"1. Based on your preferences, I recommend the ...","['Thessaloniki', 'Sibiu', 'Konya', 'Cheboksary...","1. I recommend visiting Thessaloniki, Greece. ...","[{'city': 'Thessaloniki', 's-fairness-rank': 1..."
248,mistral-instruct,prompt_47_gemini-1.5-pro-001,"['Nalchik', 'Sibiu', 'Belgrade', 'Porto', 'Tou...","1. Nalchik, Russia\n2. Sibiu, Romania\n3. Tbil...",['Porto'],1. The best place to stay in Porto for a budge...,"[{'city': 'Porto', 's-fairness-rank': 1}]"
265,mistral-instruct,prompt_44_gpt-4o-mini,"['Varna', 'Munich', 'Kahramanmaras', 'Arkhange...","1. I recommend Varna, Bulgaria. Varna is a cit...","['Munich', 'Vitoria-Gasteiz', 'Kaunas', 'Orade...","1. Munich, Germany\n2. Vitoria-Gasteiz, Spain\...","[{'city': 'Munich', 's-fairness-rank': 1}, {'c..."
270,mistral-instruct,prompt_63_gpt-4o-mini,"['Strasbourg', 'Copenhagen']","1. I recommend exploring Strasbourg, France. T...",['Copenhagen'],"1. I recommend visiting Copenhagen, Denmark. I...","[{'city': 'Copenhagen', 's-fairness-rank': 1}]"
313,mistral-instruct,prompt_39_gemini-ui,['Copenhagen'],"1. Copenhagen, Denmark\n2. Barcelona, Spain\n3...","['Copenhagen', 'Belgrade', 'Mykolaiv']","1. Copenhagen, Denmark - The sustainability (o...","[{'city': 'Copenhagen', 's-fairness-rank': 1},..."
317,mistral-instruct,prompt_20_gemini-1.5-pro-001,[],1. The capital of Romania is Bucharest.\n2. Th...,"['Syktyvkar', 'Cagliari', 'Rennes']",1. Syktyvkar is a city in Russia. The sustaina...,"[{'city': 'Syktyvkar', 's-fairness-rank': 1}, ..."
323,mistral-instruct,prompt_26_gpt-4o-mini,['Strasbourg'],"1. I recommend Strasbourg, France. Strasbourg ...",['Strasbourg'],1. I recommend Strasbourg in France.\n\nStrasb...,"[{'city': 'Strasbourg', 's-fairness-rank': 1}]"
334,mistral-instruct,prompt_26_gemini-ui,[],"1. Barcelona, Spain\n2. Amsterdam, Netherlands...","['Copenhagen', 'Santander']","1. Copenhagen, Denmark\n2. Barcelona, Spain\n3...","[{'city': 'Copenhagen', 's-fairness-rank': 1},..."
337,mistral-instruct,prompt_48_gemini-ui,['Milan'],1. I recommend Milano for your business trip i...,"['Milan', 'Rome']",1. Milan: Milan is a great choice for business...,"[{'city': 'Milan', 's-fairness-rank': 1}, {'ci..."


In 8 prompts, Llama recommends the most sustainable retrieved city as its top choice, as opposed to 20 prompts in Mistral where this is the case

On average what position does the most sustainable recommended city have? i.e. rank(min(s-fairness-rank))?

In [55]:
def find_lowest_rank(ranked_list):
    if len(ranked_list):
        min_rank = min(ranked_list, key=lambda x: x['s-fairness-rank'])['s-fairness-rank']
        # print("Min Rank:", min_rank)

        for i, cities in enumerate(ranked_list):
            # print(cities)
            if cities['s-fairness-rank'] == min_rank:
                return i+1
    return 0

In [56]:
llama_lowest_ranked = np.mean(llama_ranks['sustainability_ranks'].apply(find_lowest_rank))
mistral_lowest_ranked = np.mean(mistral_ranks['sustainability_ranks'].apply(find_lowest_rank))

In [57]:
print(llama_lowest_ranked, mistral_lowest_ranked)

1.995 1.98


Average rank of 1st recommendation

In [58]:
llama_avg_rank = np.mean(llama_ranks['sustainability_ranks'].apply(lambda x: x[0]['s-fairness-rank'] if len(x) else 0))
mistral_avg_rank = np.mean(mistral_ranks['sustainability_ranks'].apply(lambda x: x[0]['s-fairness-rank'] if len(x) else 0))

print(llama_avg_rank, mistral_avg_rank)

6.28 5.155


Mean Average S-Fairness Rank

In [59]:
def avg_rank(rec_cites):
    if len(rec_cites):
        return sum([x['s-fairness-rank'] for x in rec_cites])/len(rec_cites)
    return 0

llama_mar = np.mean(llama_ranks['sustainability_ranks'].apply(avg_rank))
mistral_mar = np.mean(mistral_ranks['sustainability_ranks'].apply(avg_rank))

print(llama_mar, mistral_mar)

6.079438492063491 5.0136964285714285


The Mean Average Sustainability Rank for Llama is slightly lower for Llama as compared to Mistral 

### For how many prompts do the responses differ when sustainability is added? 

In [60]:
def count_sustainability_responses(df, len_th, comp = 'eq'):
    count_no_common_recs = 0

    for i, row in df.iterrows():
        l1 = set(row['rec_cities'])
        l2 = set(row['rec_cities_sustainable'])

        if comp == "eq":
            if len(l1 & l2) == len_th:
                count_no_common_recs += 1
        elif comp == "lte":
            if len(l1 & l2) <= len_th:
                count_no_common_recs += 1
        elif comp == "gte":
            if len(l1 & l2) >= len_th:
                count_no_common_recs += 1


    # print("Count of records where values of C are the same across different values of A:\n", count_same_values)
    return count_no_common_recs

In [61]:
llama_none_common = count_sustainability_responses(llama, 0)
mistral_none_common = count_sustainability_responses(mistral, 0)

In [62]:
print(llama_none_common, mistral_none_common)

0 0


For no prompt are the responses completely different when sustainability is added (i.e. a totally new list of recommended cities). 

## With SAR

In [63]:
llama = combined_df[combined_df['model'] == 'llama3point1-instruct']
mistral = combined_df[combined_df['model'] == 'mistral-instruct']

In [64]:
def get_ranks(df):
    city_s_ranks = []
    for i, row in df.iterrows():
        context_path = os.path.join(sar_results_dir, row['model'], row['prompt_id'], 'cities_sustainable.json')
        with open(context_path) as f:
            context = json.load(f)

        city_rank = []
        for city in ast.literal_eval(row['rec_cities_sar']):
            for j, rec_city in enumerate(context): 
                if city == rec_city['city']:
                    city_rank.append({
                        'city': city,
                        's-fairness-rank': j+1
                    })
        city_s_ranks.append(city_rank)

    df['sustainability_ranks'] = city_s_ranks
    return df

In [65]:
llama_ranks = get_ranks(llama)

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_38824/3409777695.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sustainability_ranks'] = city_s_ranks


In [66]:
llama_ranks

,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,rec_cities_sar,response_sar,sustainability_ranks
0,llama3point1-instruct,prompt_17_gemini-ui,['Kaunas'],I recommend Kaunas in Lithuania for your expe...,['Vitoria-Gasteiz'],I recommend Vitoria-Gasteiz because it is a c...,"['Kaunas', 'Sibiu']",I recommend Kaunas and Sibiu. I recommend the...,"[{'city': 'Kaunas', 's-fairness-rank': 2}, {'c..."
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"I recommend Copenhagen, Denmark because it is...",['Copenhagen'],I recommend Copenhagen and why I recommended ...,"['Kahramanmaras', 'Nevsehir']","I recommend Baia Mare, Kahramanmaras, and Nev...","[{'city': 'Kahramanmaras', 's-fairness-rank': ..."
2,llama3point1-instruct,prompt_42_gpt-4o-mini,['Amsterdam'],"I recommend Amsterdam, Netherlands. Amsterdam...","['Nalchik', 'Varna', 'Kahramanmaras']","I recommend Nalchik, Russia because of its be...","['Nalchik', 'Arkhangelsk', 'Amsterdam']","I recommend Nalchik, Arkhangelsk, and Amsterd...","[{'city': 'Nalchik', 's-fairness-rank': 10}, {..."
3,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Kaunas', 'Thessaloniki', 'Strasbourg']","I recommend Kaunas, Lithuania. I recommend Ka...","['Kaunas', 'Oradea', 'Strasbourg', 'Thessaloni...","I recommend Kaunas, Lithuania because it is a...","['Kaunas', 'Oradea', 'Sibiu']","I recommend Kaunas, Oradea, and Sibiu. I reco...","[{'city': 'Kaunas', 's-fairness-rank': 2}, {'c..."
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,['Kaunas'],I recommend Kaunas for your medieval history ...,['Kaunas'],"I recommend Kaunas, Lithuania because it has ...","['Kaunas', 'Rennes', 'Copenhagen']","I recommend Kaunas, Rennes, and Copenhagen fo...","[{'city': 'Kaunas', 's-fairness-rank': 6}, {'c..."
...,...,...,...,...,...,...,...,...,...
195,llama3point1-instruct,prompt_29_gpt-4o-mini,['Innsbruck'],"I recommend Innsbruck, Austria. Innsbruck is ...",['Innsbruck'],"I recommend Innsbruck, Austria, for the best ...","['Innsbruck', 'Strasbourg']",I recommend Innsbruck and Strasbourg because ...,"[{'city': 'Innsbruck', 's-fairness-rank': 6}, ..."
196,llama3point1-instruct,prompt_44_gemini-ui,['Stavanger'],"I recommend Stavanger, Norway, because it off...",['Stavanger'],"I recommend Stavanger, Norway because of its ...","['Nalchik', 'Kaunas', 'Stavanger', 'Erzurum', ...","I recommend Nalchik, Kaunas, and Stavanger. I...","[{'city': 'Nalchik', 's-fairness-rank': 2}, {'..."
197,llama3point1-instruct,prompt_59_gpt-4o-mini,"['Munich', 'Thessaloniki', 'Arkhangelsk', 'Gaz...","I recommend Munich, Germany for its vibrant a...","['Munich', 'Thessaloniki', 'Gaziantep', 'Malat...",I recommend Munich and why I recommended it: ...,"['Oradea', 'Gaziantep', 'Varna']","I recommend Oradea, Gaziantep, and Varna. I r...","[{'city': 'Oradea', 's-fairness-rank': 4}, {'c..."
198,llama3point1-instruct,prompt_50_gemini-ui,"['Varna', 'Tallinn', 'London', 'Cork', 'Innsbr...","I recommend Varna, Bulgaria for your weekend ...",['Varna'],I recommend Varna because it has a great bala...,"['Tallinn', 'Cork']","I recommend Tallinn, Baia Mare, and Cork. I r...","[{'city': 'Tallinn', 's-fairness-rank': 9}, {'..."


In [67]:
mistral_ranks = get_ranks(mistral)
mistral_ranks

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_38824/3409777695.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sustainability_ranks'] = city_s_ranks


,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,rec_cities_sar,response_sar,sustainability_ranks
200,mistral-instruct,prompt_17_gemini-ui,['Budapest'],1. Paris is a city in France. Here is some inf...,"['Budapest', 'Copenhagen']","1. Budapest, Hungary\n2. Copenhagen, Denmark\n...","['Copenhagen', 'Budapest']",1. Sustainable destinations in Europe:\n* Barc...,"[{'city': 'Copenhagen', 's-fairness-rank': 3},..."
201,mistral-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"1. I recommend Copenhagen, Denmark. Copenhagen...","['Copenhagen', 'Coimbra']","1. Copenhagen, Denmark - Copenhagen is a city ...","['Kahramanmaras', 'Varna', 'Coimbra', 'Stavang...",1. I recommend Baia Mare in Romania as it has ...,"[{'city': 'Kahramanmaras', 's-fairness-rank': ..."
202,mistral-instruct,prompt_42_gpt-4o-mini,"['Amsterdam', 'Mykolaiv', 'Craiova']","1. Amsterdam, Netherlands\n2. Istanbul, Turkey...",['Amsterdam'],"1. ""What are the best things to do in Amsterda...","['Copenhagen', 'Amsterdam']","1. Copenhagen, Denmark\n2. Amsterdam, Netherla...","[{'city': 'Copenhagen', 's-fairness-rank': 1},..."
203,mistral-instruct,prompt_0_gpt-4o-mini,['Sibiu'],1. I recommend Sibiu in Romania.\n\nSibiu is a...,['Sibiu'],1. I recommend Sibiu in Romania.\n\nSibiu is a...,['Strasbourg'],1. I recommend Strasbourg. The city has a rich...,"[{'city': 'Strasbourg', 's-fairness-rank': 5}]"
204,mistral-instruct,prompt_44_gemini-1.5-pro-001,[],1. {{see\n| name | alt | url | email\n| addres...,['Prague'],"1. Option 1: Prague, Czech Republic\n* {{marke...",[],1. {{marker|typesee|nameEgapark|lat50.095572|l...,[]
...,...,...,...,...,...,...,...,...,...
395,mistral-instruct,prompt_29_gpt-4o-mini,"['Innsbruck', 'Varna', 'Kaunas', 'Kayseri', 'I...","1. I recommend recommending Innsbruck, Austria...","['Innsbruck', 'Kaunas', 'Varna', 'Kayseri', 'I...","1. Innsbruck, Austria\n2. Kaunas, Lithuania\n3...","['Varna', 'Kaunas', 'Oradea', 'Kayseri', 'Ispa...",1. The sustainability (or s-fairness) score fo...,"[{'city': 'Varna', 's-fairness-rank': 2}, {'ci..."
396,mistral-instruct,prompt_44_gemini-ui,[],1. The capital of the Czech Republic is Prague...,['Arkhangelsk'],1. What is the sustainability score for Arkhan...,[],1. 1.1\n2. 1.2\n3. 1.3\n4. 1.4\n5. 1.5\n6. 1.6...,[]
397,mistral-instruct,prompt_59_gpt-4o-mini,"['Malatya', 'Varna', 'Mykolaiv', 'Munich', 'Ar...","1. I recommend visiting Malatya, Turkey. The c...","['Mykolaiv', 'Munich', 'Varna', 'Craiova', 'Ar...",1. I recommend visiting Mykolaiv in Ukraine. M...,"['Oradea', 'Varna', 'Craiova']",1. I recommend visiting Oradea in Romania. The...,"[{'city': 'Oradea', 's-fairness-rank': 4}, {'c..."
398,mistral-instruct,prompt_50_gemini-ui,[],1. Option 1:\n\n* {{eat\n| name\n| url\n| emai...,"['Innsbruck', 'Antalya', 'Cork', 'London']",1. The sustainability score for the city of In...,['London'],1. The sustainability (or s-fairness) score fo...,"[{'city': 'London', 's-fairness-rank': 2}]"


In [68]:
def is_lowest_rank(ranked_list):
    if len(ranked_list):
        min_rank = min(ranked_list, key=lambda x: x['s-fairness-rank'])['s-fairness-rank']

        if ranked_list[0]['s-fairness-rank'] == min_rank:
            return 1
    return 0

llama_min_ranks = sum(llama_ranks['sustainability_ranks'].apply(is_lowest_rank))
mistral_min_ranks = sum(mistral_ranks['sustainability_ranks'].apply(is_lowest_rank))

print(llama_min_ranks, mistral_min_ranks)

85 73


In [69]:
def most_sustainable(ranked_list):
    if len(ranked_list):
        if ranked_list[0]['s-fairness-rank'] == 1:
            return True
    return False

In [70]:
def is_top_choice_most_sustainable(ranked_list):
    if len(ranked_list):
        if ranked_list[0]['s-fairness-rank'] == 1:
            return 1
    return 0

In [71]:
l_top_choice = sum(llama_ranks['sustainability_ranks'].apply(is_top_choice_most_sustainable))
m_top_choice = sum(mistral_ranks['sustainability_ranks'].apply(is_top_choice_most_sustainable))

print(l_top_choice, m_top_choice)

21 15


In [72]:
llama_avg_rank = np.mean(llama_ranks['sustainability_ranks'].apply(lambda x: x[0]['s-fairness-rank'] if len(x) else 0))
mistral_avg_rank = np.mean(mistral_ranks['sustainability_ranks'].apply(lambda x: x[0]['s-fairness-rank'] if len(x) else 0))

print(llama_avg_rank, mistral_avg_rank)

5.085 5.015


In [73]:
def avg_rank(rec_cites):
    if len(rec_cites):
        return sum([x['s-fairness-rank'] for x in rec_cites])/len(rec_cites)
    return 0

llama_mar = np.mean(llama_ranks['sustainability_ranks'].apply(avg_rank))
mistral_mar = np.mean(mistral_ranks['sustainability_ranks'].apply(avg_rank))

print(llama_mar, mistral_mar)

5.2435615079365085 4.787412698412698


In [74]:
def most_sustainable(ranked_list):
    if len(ranked_list):
        if ranked_list[0]['s-fairness-rank'] == 1:
            return True
    return False

In [75]:
m_sar = mistral_ranks[(mistral_ranks['sustainability_ranks'].apply(most_sustainable))]
m_sar

,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,rec_cities_sar,response_sar,sustainability_ranks
202,mistral-instruct,prompt_42_gpt-4o-mini,"['Amsterdam', 'Mykolaiv', 'Craiova']","1. Amsterdam, Netherlands\n2. Istanbul, Turkey...",['Amsterdam'],"1. ""What are the best things to do in Amsterda...","['Copenhagen', 'Amsterdam']","1. Copenhagen, Denmark\n2. Amsterdam, Netherla...","[{'city': 'Copenhagen', 's-fairness-rank': 1},..."
229,mistral-instruct,prompt_10_gpt-4o-mini,['Arkhangelsk'],"1. I recommend Arkhangelsk, Russia. Arkhangels...",['Arkhangelsk'],"1. I recommend Arkhangelsk, Russia. Arkhangels...","['Arkhangelsk', 'Oradea', 'Simferopol', 'Kahra...",1. I recommend Arkhangelsk as a romantic getaw...,"[{'city': 'Arkhangelsk', 's-fairness-rank': 1}..."
237,mistral-instruct,prompt_5_gpt-4o-mini,"['Munich', 'Nalchik', 'Gaziantep', 'Oradea', '...",1. The city of Munich in Germany is a great de...,"['Erzurum', 'Maastricht', 'Nalchik', 'Gaziante...","1. I recommend Istanbul, Turkey. Istanbul has ...",['Munich'],"1. I recommend visiting Munich, Germany in May...","[{'city': 'Munich', 's-fairness-rank': 1}]"
239,mistral-instruct,prompt_47_gemini-ui,[],"1. Based on your preferences, I recommend the ...","['Thessaloniki', 'Sibiu', 'Konya', 'Cheboksary...","1. I recommend visiting Thessaloniki, Greece. ...",['Thessaloniki'],"1. I recommend visiting Thessaloniki, Greece i...","[{'city': 'Thessaloniki', 's-fairness-rank': 1}]"
267,mistral-instruct,prompt_51_gpt-4o-mini,[],"1. I recommend Varkala, India for outdoor acti...","['Varna', 'Copenhagen', 'Erzurum', 'Mykolaiv',...","1. Varna, Bulgaria: This coastal city has beau...","['Copenhagen', 'Arkhangelsk', 'Kahramanmaras',...","1. I recommend visiting Copenhagen, Denmark. T...","[{'city': 'Copenhagen', 's-fairness-rank': 1},..."
270,mistral-instruct,prompt_63_gpt-4o-mini,"['Strasbourg', 'Copenhagen']","1. I recommend exploring Strasbourg, France. T...",['Copenhagen'],"1. I recommend visiting Copenhagen, Denmark. I...","['Copenhagen', 'Hamburg', 'Innsbruck', 'Klagen...","1. Copenhagen, Denmark\n2. Hamburg, Germany\n3...","[{'city': 'Copenhagen', 's-fairness-rank': 1},..."
291,mistral-instruct,prompt_3_gpt-4o-mini,['Strasbourg'],"1. I recommend Strasbourg, France. This city i...",['Sibiu'],1. I recommend Sibiu in Romania. Sibiu is a ci...,"['Strasbourg', 'Sibiu', 'Copenhagen', 'Plovdiv...",1. I recommend Strasbourg for your autumn trip...,"[{'city': 'Strasbourg', 's-fairness-rank': 1},..."
301,mistral-instruct,prompt_10_gemini-1.5-pro-001,[],1. What is the name of the city in Russia?\n2....,"['Aalborg', 'Nalchik', 'Kazan', 'London']","1. Barcelona, Spain\n2. Sapporo, Japan\n3. Aal...","['Nalchik', 'Aalborg', 'London', 'Kazan']",1. The city of Viborg is in Denmark. The susta...,"[{'city': 'Nalchik', 's-fairness-rank': 1}, {'..."
313,mistral-instruct,prompt_39_gemini-ui,['Copenhagen'],"1. Copenhagen, Denmark\n2. Barcelona, Spain\n3...","['Copenhagen', 'Belgrade', 'Mykolaiv']","1. Copenhagen, Denmark - The sustainability (o...","['Copenhagen', 'Belgrade', 'Arkhangelsk', 'Nev...","1. Copenhagen, Denmark - Sustainability (or s-...","[{'city': 'Copenhagen', 's-fairness-rank': 1},..."
317,mistral-instruct,prompt_20_gemini-1.5-pro-001,[],1. The capital of Romania is Bucharest.\n2. Th...,"['Syktyvkar', 'Cagliari', 'Rennes']",1. Syktyvkar is a city in Russia. The sustaina...,"['Syktyvkar', 'Cagliari', 'Gaziantep', 'Rennes']",1. Option 1: Tromsø is a city in Norway.\n2. O...,"[{'city': 'Syktyvkar', 's-fairness-rank': 1}, ..."


In [76]:
l_sar = llama_ranks[(llama_ranks['sustainability_ranks'].apply(most_sustainable))]
l_sar

,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,rec_cities_sar,response_sar,sustainability_ranks
20,llama3point1-instruct,prompt_55_gemini-ui,['Varna'],I recommend Varna is a city in Bulgaria.Here ...,"['Copenhagen', 'Samsun', 'Burgas', 'Varna', 'S...","I recommend Copenhagen, Denmark, because it i...","['Varna', 'Santander']",I recommend Varna and Santander for a family-...,"[{'city': 'Varna', 's-fairness-rank': 1}, {'ci..."
21,llama3point1-instruct,prompt_17_gemini-1.5-pro-001,"['Nalchik', 'Perugia']","I recommend Nalchik, Russia for a yoga retrea...",['Nalchik'],I recommend Nalchik as the best city for a yo...,"['Arkhangelsk', 'Perugia']",I recommend Arkhangelsk and Perugia for a yog...,"[{'city': 'Arkhangelsk', 's-fairness-rank': 1}..."
36,llama3point1-instruct,prompt_47_gpt-4o-mini,['Strasbourg'],I recommend Strasbourg for its Christmas mark...,"['Strasbourg', 'Varna', 'Kaunas', 'Valencia', ...",I recommend Strasbourg in France for its beau...,"['Varna', 'Strasbourg', 'Kaunas']","I recommend Varna, Strasbourg and Kaunas as t...","[{'city': 'Varna', 's-fairness-rank': 1}, {'ci..."
57,llama3point1-instruct,prompt_37_gemini-ui,['Thessaloniki'],"I recommend Thessaloniki, Greece. I recommend...",['Budapest'],"I recommend Budapest, Hungary, in June. I rec...","['Thessaloniki', 'Izmir', 'Budapest']","I recommend Thessaloniki, Izmir and Budapest ...","[{'city': 'Thessaloniki', 's-fairness-rank': 1..."
65,llama3point1-instruct,prompt_44_gpt-4o-mini,['Munich'],"I recommend Munich, Germany because it has a ...",['Kaunas'],I recommend Kaunas as a city that celebrates ...,"['Munich', 'Kaunas', 'Oradea']","I recommend Munich, Kaunas and Oradea. I reco...","[{'city': 'Munich', 's-fairness-rank': 1}, {'c..."
66,llama3point1-instruct,prompt_21_gpt-4o-mini,['Vitoria-Gasteiz'],I recommend attending the Azkena Rock Festiva...,['Vitoria-Gasteiz'],I recommend Vitoria-Gasteiz for its vibrant a...,"['Munich', 'Rome', 'Warsaw', 'London', 'Ankara']","I recommend Munich, Rome, Warsaw, London and ...","[{'city': 'Munich', 's-fairness-rank': 1}, {'c..."
68,llama3point1-instruct,prompt_61_gemini-ui,['Plovdiv'],I recommend Plovdiv because it offers a great...,"['Nalchik', 'Craiova', 'Plovdiv']",I recommend Nalchik. I recommend Nalchik beca...,"['Oradea', 'Plovdiv', 'Craiova']","I recommend Oradea, Plovdiv, and Craiova. Ora...","[{'city': 'Oradea', 's-fairness-rank': 1}, {'c..."
69,llama3point1-instruct,prompt_12_gemini-1.5-pro-001,['Paris'],"I recommend Paris, France, in September. Pari...",['Paris'],I recommend Paris for its culinary expertise ...,"['Dijon', 'Paris', 'Helsinki', 'London', 'Rome']","I recommend Dijon, Paris, Helsinki, London, a...","[{'city': 'Dijon', 's-fairness-rank': 1}, {'ci..."
91,llama3point1-instruct,prompt_3_gpt-4o-mini,"['Sibiu', 'Plovdiv', 'Strasbourg']","I recommend Sibiu, Romania. I recommend Sibiu...","['Sibiu', 'Mykolaiv', 'Strasbourg', 'Craiova']","I recommend Sibiu, Romania, because it is a c...","['Strasbourg', 'Sibiu', 'Plovdiv']","I recommend Strasbourg, Sibiu, and Plovdiv. I...","[{'city': 'Strasbourg', 's-fairness-rank': 1},..."
93,llama3point1-instruct,prompt_7_gemini-1.5-pro-001,"['Paris', 'Rome', 'Bordeaux']",I recommend Paris and why you recommended it:...,['Paris'],"I recommend Paris, because it is one of the g...","['Kahramanmaras', 'Cluj-Napoca', 'Paris']","I recommend Kahramanmaras, Cluj-Napoca and Pa...","[{'city': 'Kahramanmaras', 's-fairness-rank': ..."
